# Generate models with numpyro

- my previous model fits
https://github.com/sylvielsstfr/FitDiffAtmo/blob/main/docs/notebooks/fitdiffatmo/test_numpyro_orderedict_diffatmemul_5params_P_pwv_oz_tau_beta.ipynb

- dsps tuto https://dsps.readthedocs.io/en/latest/dsps_quickstart.html

- author Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNRS
- CC: kernel conda_jax0235_py310
- creation date : 2023-11-10
- update : 2023-11-10


| computer | processor | kernel              |    date     |
| --- | --- | --- | --- |
| CC       | CPU       | conda_jax0325_py310 | 2023-11-10  |
| macbookpro | CPU | conda_jaxcpu_dsps_py310 | 2023-11-10  | 


libraries 
=========

jax
---

- jaxlib-0.3.25+cuda11.cudnn82
- jaxopt
- optax
- corner
- arviz
- numpyro
- graphviz

sps
---

- fsps
- prospect
- dsps
- diffstar
- diffmah



(conda_jax0325_py310) 
`/pbs/throng/lsst/users/dagoret/desc/StellarPopulationSynthesis>pip list | grep` 

| lib | version |
|--- | --- | 
|jax  |                         0.4.20 |
|jaxlib |                       0.4.20 |
|jaxopt  |                      0.8.2 |



- see example https://github.com/sylvielsstfr/FitDiffAtmo/blob/main/docs/notebooks/fitdiffatmo/test_numpyro_orderedict_diffatmemul_5params_P_pwv_oz_tau_beta.ipynb

In [ ]:
import numpy as np
import jax
import jax.numpy as jnp
from jax.scipy.special import logsumexp
jax.config.update("jax_enable_x64", True)

import numpyro
from numpyro import optim
from numpyro.diagnostics import print_summary
import numpyro.distributions as dist
from numpyro.distributions import constraints
from numpyro.infer import MCMC, HMC, NUTS, SVI, Trace_ELBO
from numpyro.infer.autoguide import AutoBNAFNormal, AutoMultivariateNormal
from numpyro.infer.reparam import NeuTraReparam
from numpyro.handlers import seed, trace, condition

import matplotlib as mpl
from matplotlib import pyplot as plt

import corner
import arviz as az
mpl.rcParams['font.size'] = 15
mpl.rcParams["figure.figsize"] = [8, 8]

import os
import jax
import jax.numpy as jnp

In [ ]:
import h5py
import pandas as pd
import numpy as np
import os
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt
%matplotlib inline
from collections import OrderedDict
import re
import matplotlib.gridspec as gridspec
from sklearn.gaussian_process import GaussianProcessRegressor, kernels

In [ ]:
plt.rcParams["figure.figsize"] = (12,6)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [ ]:
from diffstar import sfh_singlegal
from dsps.cosmology import age_at_z, DEFAULT_COSMOLOGY

In [ ]:
from dsps import load_ssp_templates

In [ ]:
from diffstar.defaults import DEFAULT_MAH_PARAMS
from diffstar.defaults import DEFAULT_MS_PARAMS
from diffstar.defaults import DEFAULT_Q_PARAMS

In [ ]:
from dsps import calc_rest_sed_sfh_table_lognormal_mdf
from dsps import calc_rest_sed_sfh_table_met_table

In [ ]:
from dsps.dust.att_curves import  sbl18_k_lambda, RV_C00,_frac_transmission_from_k_lambda

### Very important add the lib to load the data

In [ ]:
import sys
sys.path.append("../../lib")

In [ ]:
from fit_params_fors2 import U_FNU,U_FL,ConvertFlambda_to_Fnu,flux_norm,ordered_keys,Fors2DataAcess

In [ ]:
lambda_red = 6231
lambda_width = 50
lambda_sel_min = lambda_red-lambda_width /2.
lambda_sel_max = lambda_red+lambda_width /2.

## Load fors2 spectra

In [ ]:
input_file_h5  = '../../data/fors2sl/FORS2spectraGalexKidsPhotom.hdf5'
fors2 = Fors2DataAcess(input_file_h5)
list_of_keys = fors2.get_list_of_groupkeys()
list_of_attributes = fors2.get_list_subgroup_keys()

### sort keys

In [ ]:
list_of_keys = np.array(list_of_keys)
list_of_keysnum = [ int(re.findall("SPEC(.*)",specname)[0]) for specname in  list_of_keys ]
sorted_indexes = np.argsort(list_of_keysnum)
list_of_keys = list_of_keys[sorted_indexes]
df_info = pd.DataFrame(columns=list_of_attributes)
all_df = []

### Retrieve Fors2 spectra in order 

In [ ]:
for idx,key in enumerate(list_of_keys):
    attrs = fors2.getattribdata_fromgroup(key)
    spectr = fors2.getspectrum_fromgroup(key)
    df_info.loc[idx] = [*attrs.values()] # hope the order of attributes is kept
    df = pd.DataFrame({"wl":spectr["wl"],"fnu":spectr["fnu"]})
    all_df.append(df)
    

In [ ]:
df_info.reset_index(drop=True, inplace=True) 
df_info = df_info[ordered_keys]
df_info

### Select those spectra having a good match with Galex

In [ ]:
df_info.hist("asep_galex",bins=100,color="b")
plt.axvline(5,c="k")
df_info.hist("asep_kids",bins=100,color='b')

### Select those spectra that have angular correlation with Galex

In [ ]:
df = df_info[df_info["asep_galex"] <= 5]

## Plot Spectra

In [ ]:
lambda_FUV = 1528.
lambda_NUV = 2271.
lambda_U = 3650.
lambda_B = 4450.
lambda_G = 4640.
lambda_R = 5580.
lambda_I = 8060.
lambda_Z = 9000.
lambda_Y = 10200.
lambda_J = 12200.
lambda_H = 16300.
lambda_K = 21900.
lambda_L = 34500.

WL = [lambda_FUV, lambda_NUV, lambda_B, lambda_G, lambda_R ,lambda_I, lambda_Z, lambda_Y, lambda_J, lambda_H, lambda_K ]
FilterTag = ['FUV','NUV','B','G','R','I','Z','Y','J','H','Ks']

In [ ]:
def PlotFilterTag(ax,fluxlist):
    goodfl = fluxlist[np.isfinite(fluxlist)]
    ymin = np.mean(goodfl)
    dy=ymin/5
   
    for idx,flux in enumerate(fluxlist):
        if np.isfinite(flux):
            #ax.text(WL[idx],flux, FilterTag[idx],fontsize=10,ha='center', va='bottom')
                     
            fl = flux - dy
            if fl <0:
                fl += 2*dy
            ax.text(WL[idx],fl, FilterTag[idx],fontsize=12,color="g",weight='bold',ha='center', va='bottom')
            

In [ ]:
df['index0'] = df.index
df = df.reset_index()

## Fit Gaussian process to remove abs line

In [ ]:
kernel = kernels.RBF(0.5, (8000, 10000.0))
gp = GaussianProcessRegressor(kernel=kernel ,random_state=0)

#### Select the spectrum

In [ ]:
#selected_spectrum_number = 411
selected_spectrum_number = 560

dict_normalisation_factor = {}
dict_normalisation_factor[411] = 160.
dict_normalisation_factor[560] = 12.

In [ ]:
# loop on rows to select the spectrum selected above
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
for idx,row in df.iterrows():
    
    print(idx," == ",row["name"],row["num"],row["index0"])
    idx0 = row["index0"] # index for the spectrum
    specname  = row["name"]
    specnum = row["num"]
    if specnum < selected_spectrum_number:
        continue
    if specnum > selected_spectrum_number:
        break
    
    
    mags = [ row["fuv_mag"], row["nuv_mag"], row['MAG_GAAP_u'], row['MAG_GAAP_g'], row['MAG_GAAP_r'], row['MAG_GAAP_i'], row['MAG_GAAP_Z'], row['MAG_GAAP_Y'], 
            row['MAG_GAAP_J'], row['MAG_GAAP_H'],row['MAG_GAAP_Ks'] ]
    
    magserr = [ row["fuv_magerr"], row["nuv_magerr"], row['MAGERR_GAAP_u'], row['MAGERR_GAAP_g'], row['MAGERR_GAAP_r'], row['MAGERR_GAAP_i'], row['MAGERR_GAAP_Z'], row['MAGERR_GAAP_Y'], 
            row['MAGERR_GAAP_J'], row['MAGERR_GAAP_H'],row['MAGERR_GAAP_Ks'] ]
    
    mfluxes = [ 10**(-0.4*m) for m in mags ]
    mfluxeserr = []
    
    for f,em in zip(mfluxes,magserr):
        ferr = 0.4*np.log(10)*em*f
        mfluxeserr.append(ferr)
        
    mfluxes = np.array(mfluxes)
    mfluxeserr = np.array(mfluxeserr)
    
    fluxes =  [ row["fuv_flux"], row["nuv_flux"], row['FLUX_GAAP_u'], row['FLUX_GAAP_g'], row['FLUX_GAAP_r'], row['FLUX_GAAP_i'], row['FLUX_GAAP_Z'], row['FLUX_GAAP_Y'], 
            row['FLUX_GAAP_J'], row['FLUX_GAAP_H'],row['FLUX_GAAP_Ks'] ]
    
    fluxeserr =  [ row["fuv_fluxerr"], row["nuv_fluxerr"], row['FLUXERR_GAAP_u'], row['FLUXERR_GAAP_g'], row['FLUXERR_GAAP_r'], row['FLUXERR_GAAP_i'], row['FLUXERR_GAAP_Z'], row['FLUX_GAAP_Y'], 
            row['FLUXERR_GAAP_J'], row['FLUXERR_GAAP_H'],row['FLUXERR_GAAP_Ks'] ]
    
    ##############################################################
    # The top image
    ######################################
    fig = plt.figure(figsize=(10,14),tight_layout=True)
    gs = gridspec.GridSpec(3, 1,height_ratios=[3, 1, 3])
    
    
    #top image : original flux
    ax= fig.add_subplot(gs[0, 0])
    
    z = row["redshift"]
    zobs = z
    asep_fg = row['asep_galex']
    rmag = row["Rmag"]
    asep_fk = row['asep_kids']
    
  
    the_lines = row['lines']
    print("the_lines = ",the_lines)
    all_elements  = the_lines.decode().split(",")
    
    
    the_label_data = f" FORS2 : z = {z:.3f} , Rmag = {rmag:.1f} mag,  angular sep (arcsec) f-g : {asep_fg:.3f}, f-k : {asep_fk:.3f}" 
    the_wl = all_df[idx0]["wl"].values
    the_fnu = all_df[idx0]["fnu"].values*10**(-0.4*mags[4])    
    ax.plot(the_wl,the_fnu,'b-',label=the_label_data)
     
    X = the_wl
    Y = the_fnu
    
    gp.fit(X[:, None], Y)
    xfit = np.linspace(X.min(),X.max())
    yfit, yfit_err = gp.predict(xfit[:, None], return_std=True)
    ax.plot(xfit, yfit, '-', color='cyan')
    ax.fill_between(xfit, yfit -  yfit_err, yfit +  yfit_err, color='gray', alpha=0.3)
     
    ax2 = ax.twinx()
    ax2.errorbar(WL,mfluxes,yerr=mfluxeserr,fmt='o',color="r",ecolor="r",ms=5,label='Galex (UV) + Kids (optics) +Viking (IR)')
    PlotFilterTag(ax2,mfluxes)
    #ax2.errorbar(WL, fluxes, yerr=fluxes, xerr=None, fmt='o', color="g",ecolor="g")
    ax2.legend(loc="lower right")
    
    ax.set_xlabel("$\lambda  (\AA)$ ")
    title_data = f"FORS2 : {idx}): {idx0} name = {specname}"
    ax.set_title(title_data)
    ax.set_ylabel("flux (maggies)")
    ax.legend(loc="upper right")
    ax.grid()
    
    the_max1 = np.max(the_fnu)
    goodmags = mfluxes[np.isfinite(mfluxes)]
    the_max2 = np.max(goodmags)
    the_max = 1.1*max(the_max1,the_max2)

    ax.set_ylim(0,the_max)
    ax2.set_ylim(0,the_max)

    ax.set_xlim(1000.,25000.)
    ##################
    # Middle image : Fit a gaussian process and compute the residuals
    ##################
    ax3 = fig.add_subplot(gs[1, 0])
    DeltaY,DeltaEY = Y - gp.predict(X[:, None], return_std=True)
    ax3.plot(X,DeltaY,'b')
    ax3.set_xlim(1000.,25000.)
    ax3.grid()
    ax3.set_xlabel("$\lambda  (\AA)$ ")
    
    background = np.sqrt(np.median(DeltaY**2))
    indexes_toremove = np.where(np.abs(DeltaY)> 8 * background)[0]
    
    for index in indexes_toremove:
        ax3.axvline(X[index],color='k')
    
    #########################    
    # Bottom image : resuting image after removing emission lines
    #########################
    
    ax4 = fig.add_subplot(gs[2, 0])
    
    Xclean = np.delete(X,indexes_toremove)
    Yclean  = np.delete(Y,indexes_toremove)
    
    ax4.plot(Xclean, Yclean,'b-',label=the_label_data)
    ax4.set_ylim(0,the_max)
    ax4.set_xlim(1000.,25000.)
    ax4.grid()
    ax4.set_xlabel("$\lambda  (\AA)$ ")
    
    ax5 = ax4.twinx()
    ax5.errorbar(WL,mfluxes,yerr=mfluxeserr,fmt='o',color="r",ecolor="r",ms=5,label='Galex (UV) + Kids (optics) +Viking (IR)')
    ax5.legend(loc="lower right")
    ax5.set_ylim(0,the_max)
    PlotFilterTag(ax5,mfluxes)
    
    ax4.set_ylabel("flux (maggies)")
    ax4.legend(loc="upper right")
    
    spectr = fors2.getspectrumcleanedemissionlines_fromgroup(specname,gp)

    the_wl = all_df[idx0]["wl"].values
    the_fnu = all_df[idx0]["fnu"].values*10**(-0.4*mags[4])
    
    ax5.plot(the_wl,the_fnu,color="grey",lw=0.25,label="original")

    
    plt.show()

# Model of a Galaxy

## Load SED templates from DSPS 

In [ ]:

#ssp_data = load_ssp_templates(fn='tempdata_v2.h5')
ssp_data = load_ssp_templates(fn='../examples_dsps_diffstar_diffmah/tempdata.h5')

print(ssp_data._fields)

print('ssp_data : ssp_lgmet.shape = {}'.format(ssp_data.ssp_lgmet.shape))
print('ssp_data : ssp_lg_age_gyr.shape = {}'.format(ssp_data.ssp_lg_age_gyr.shape))
print('ssp_data : ssp_wave.shape = {}'.format(ssp_data.ssp_wave.shape))
print('ssp_data :ssp_flux.shape = {}'.format(ssp_data.ssp_flux.shape))

## Parameters of the Galaxy spectrum model

In [ ]:
print("SFR : 13 parameters")

In [ ]:
#mah_params = (lgm0, logtc, early_index, late_index)
print('MAH 4 params (lgm0, logtc, early_index, late_index) : ',DEFAULT_MAH_PARAMS)

In [ ]:
#ms_params = (lgmcrit, lgy_at_mcrit, indx_lo, indx_hi, tau_dep)
print('MS 5 params (lgmcrit, lgy_at_mcrit, indx_lo, indx_hi, tau_dep) : ' ,DEFAULT_MS_PARAMS)

In [ ]:
# q_params (lg_qt, qlglgdt, lg_drop, lg_rejuv)
print('Q 4 params (lg_qt, qlglgdt, lg_drop, lg_rejuv) : ',DEFAULT_Q_PARAMS)

In [ ]:
#sfh_singlegal?

### Calculate age distribution

In [ ]:
today_gyr = 13.8 
tarr = np.linspace(0.1, today_gyr, 100)

In [ ]:
sfh_gal = sfh_singlegal(tarr, DEFAULT_MAH_PARAMS, DEFAULT_MS_PARAMS, DEFAULT_Q_PARAMS)

In [ ]:
fig, ax = plt.subplots(1, 1)
ylim = ax.set_ylim(1e-3, 50)
yscale = ax.set_yscale('log')

__=ax.plot(tarr, sfh_gal, '--', color='k',label='sfh_gal')
ax.set_title("Simulated Star Formation History (SFH) (log scale)")
xlabel = ax.set_xlabel(r'${\rm cosmic\ time\ [Gyr]}$')
ylabel = ax.set_ylabel(r'${\rm SFR\ [M_{\odot}/yr]}$')
ax.grid()

In [ ]:
fig, ax = plt.subplots(1, 1)
ylim = ax.set_ylim(0, 1)
#yscale = ax.set_yscale('log')

__=ax.plot(tarr, sfh_gal, '--', color='k',label='sfh_gal')
ax.set_title("Simulated Star Formation History (SFH) (lin scale)")
xlabel = ax.set_xlabel(r'${\rm cosmic\ time\ [Gyr]}$')
ylabel = ax.set_ylabel(r'${\rm SFR\ [M_{\odot}/yr]}$')
ax.grid()

## Must clear SFR after 

In [ ]:
t_obs = age_at_z(zobs, *DEFAULT_COSMOLOGY) # age of the universe in Gyr at z_obs
t_obs = t_obs[0] # age_at_z function returns an array, but SED functions accept a float for this argument

In [ ]:
t_obs

In [ ]:
sfh_gal = jnp.where(tarr<t_obs, sfh_gal, 0)

In [ ]:
fig, ax = plt.subplots(1, 1)
ylim = ax.set_ylim(0, 1)
#yscale = ax.set_yscale('log')

__=ax.plot(tarr, sfh_gal, '--', color='k',label='sfh_gal')
ax.set_title("Simulated Star Formation History (SFH) (lin scale)")
xlabel = ax.set_xlabel(r'${\rm cosmic\ time\ [Gyr]}$')
ylabel = ax.set_ylabel(r'${\rm SFR\ [M_{\odot}/yr]}$')
ax.grid()

# Draw galaxy parameters

In [ ]:
#mah_params_galpop = np.tile(DEFAULT_MAH_PARAMS, n_gals)
#mah_params_galpop = mah_params_galpop.reshape((n_gals, -1))

#ms_params_galpop = np.tile(DEFAULT_MS_PARAMS, n_gals)
#ms_params_galpop = ms_params_galpop.reshape((n_gals, -1))

#q_params_galpop = np.tile(DEFAULT_Q_PARAMS, n_gals)
#q_params_galpop = q_params_galpop.reshape((n_gals, -1))

#ms_noise = np.random.normal(
#    loc=0, scale=0.25, size=(n_gals, DEFAULT_MS_PARAMS.size))

#ms_params_galpop = ms_params_galpop + ms_noise

#q_noise = np.random.normal(
#    loc=0, scale=0.1, size=(n_gals, DEFAULT_Q_PARAMS.size))

#q_params_galpop = q_params_galpop + q_noise

### MAH parameters

In [ ]:
#mah_params = (lgm0, logtc, early_index, late_index)
DEFAULT_MAH_PARAMS

In [ ]:
mah_paramnames = ["MAH_lgmO","MAH_logtc","MAH_early_index","MAH_late_index"]

### MS parameters

In [ ]:
#ms_params = (lgmcrit, lgy_at_mcrit, indx_lo, indx_hi, tau_dep)
DEFAULT_MS_PARAMS

In [ ]:
ms_paramnames = ["MS_lgmcrit", "MS_lgy_at_mcrit", "MS_indx_lo", "MS_indx_hi", "MS_tau_dep"]

### Q parameters

In [ ]:
# q_params (lg_qt, qlglgdt, lg_drop, lg_rejuv)
DEFAULT_Q_PARAMS

In [ ]:
q_paramnames = ["Q_lg_qt", "Q_qlglgdt", "Q_lg_drop", "Q_lg_rejuv"]

### Dust parameters

In [ ]:
Av= 1
uv_bump_ampl = 3.0
plaw_slope = -0.25

In [ ]:
DEFAULT_DUST_PARAMS = [Av, uv_bump_ampl, plaw_slope]
dust_paramnames = ["Av", "uv_bump", "plaw_slope"]

## Combine parameters

In [ ]:
rng_key = jax.random.PRNGKey(1)
rng_key, rng_key0, rng_key1, rng_key2 = jax.random.split(rng_key, 4)

In [ ]:
defaults_params = [DEFAULT_MAH_PARAMS,DEFAULT_MS_PARAMS,DEFAULT_Q_PARAMS,DEFAULT_DUST_PARAMS]
param_names = [mah_paramnames,ms_paramnames,q_paramnames,dust_paramnames]
param_scales = [3, 0.25,0.1,2.]

In [ ]:
dict_param_mah_true = OrderedDict([(mah_paramnames[0],DEFAULT_MAH_PARAMS[0]),
                                         (mah_paramnames[1],DEFAULT_MAH_PARAMS[1]),
                                         (mah_paramnames[2],DEFAULT_MAH_PARAMS[2]),
                                         (mah_paramnames[3],DEFAULT_MAH_PARAMS[3])
                                         ])
dict_param_mah_true_selected = OrderedDict([(mah_paramnames[0],DEFAULT_MAH_PARAMS[0]),
                                         ])

In [ ]:
d = dist.Uniform(DEFAULT_MAH_PARAMS[0]-param_scales[0],DEFAULT_MAH_PARAMS[0]+param_scales[3])
data = d.sample(rng_key0,sample_shape=(500,))

In [ ]:
fig, ax = plt.subplots(1, 1,figsize=(3,2))
ax.hist(data,bins=50);
ax.set_title(mah_paramnames[0])

In [ ]:
dict_param_ms_true = OrderedDict([(ms_paramnames[0],DEFAULT_MS_PARAMS[0]),
                                         (ms_paramnames[1],DEFAULT_MS_PARAMS[1]),
                                         (ms_paramnames[2],DEFAULT_MS_PARAMS[2]),
                                         (ms_paramnames[3],DEFAULT_MS_PARAMS[3]),
                                         (ms_paramnames[4],DEFAULT_MS_PARAMS[4])])

In [ ]:
fig, axes = plt.subplots(2, 3,figsize=(15,6))
list_of_keys = list(dict_param_ms_true.keys())
scale = param_scales[1]
for idx, ax in enumerate(axes.flat):
    if idx>= len(list_of_keys):
        break
    d = dist.Normal(0,scale)
    data = DEFAULT_MS_PARAMS[idx] + d.sample(rng_key1,sample_shape=(500,))
    ax.hist(data,bins=50);
    ax.set_title(ms_paramnames[idx])
                    
    

plt.tight_layout()    

In [ ]:
param_scales[1]

In [ ]:
dict_param_q_true = OrderedDict([(q_paramnames[0],DEFAULT_Q_PARAMS[0]),
                                         (q_paramnames[1],DEFAULT_Q_PARAMS[1]),
                                         (q_paramnames[2],DEFAULT_Q_PARAMS[2]),
                                         (q_paramnames[3],DEFAULT_Q_PARAMS[3])])

In [ ]:
dict_param_dust_true = OrderedDict([(dust_paramnames[0],DEFAULT_DUST_PARAMS[0]),
                                         (dust_paramnames[1],DEFAULT_DUST_PARAMS[1]),
                                         (dust_paramnames[2],DEFAULT_DUST_PARAMS[2])])
dict_param_dust_true_selected = OrderedDict([(dust_paramnames[0],DEFAULT_DUST_PARAMS[0])])

### Group all parameters

In [ ]:
params_true = dict_param_mah_true_selected
params_true.update(dict_param_ms_true)
params_true.update(dict_param_q_true)
params_true.update(dict_param_dust_true_selected)

In [ ]:
params_true

## Bayesian modelling

In [ ]:
def galaxymodel(wls,Fobs=None,defaultsparamval = defaults_params,
                param_names = param_names,
                param_scales = param_scales,
                z_obs = zobs,
                sigma= 1.0):
    """
    Models of Galaxy spectrum at rest
    
    :param wls: array of input spectrum wavelength 
    :type wls: float in Angstrom
    
    mah_params parameters
    mah_params = (lgm0, logtc, early_index, late_index)
    
    ms_params : ndarray, shape (5, )
    By default the input ms_params will be interpreted as standard diffstar params:
        ms_params = (lgmcrit, lgy_at_mcrit, indx_lo, indx_hi, tau_dep)
        
    q_params : ndarray, shape (4, )
    By default the input q_params will be interpreted as standard diffstar params:
        (lg_qt, qlglgdt, lg_drop, lg_rejuv)
   
    z_obs :  redshift of observation
    
    """
    NgrpParams = len(defaultsparamval)
    
    # loop on the group of parameters
    for igrp in range(NgrpParams):
        scale =  param_scales[igrp]
        val0 = defaultsparamval[igrp]
        names = param_names[igrp]
        
        if igrp==0:
            lgm0 = numpyro.sample(names[0],dist.Uniform(val0[0]-scale,val0[0]+scale))
            dict_param_mah = OrderedDict([(names[0],lgm0)])
            list_param_mah = [lgm0, val0[1],val0[2],val0[3] ]
                                         
        # defines the parameters
        if igrp == 1:
            lgmcrit = numpyro.sample(names[0],dist.Normal(val0[0], scale))
            lgy_at_mcrit = numpyro.sample(names[1],dist.Normal(val0[1], scale))
            indx_lo = numpyro.sample(names[2],dist.Normal(val0[2], scale))
            indx_hi = numpyro.sample(names[3],dist.Normal(val0[3], scale))
            tau_dep = numpyro.sample(names[4],dist.Normal(val0[4], scale))
            
        
            dict_param_ms = OrderedDict([(names[0],lgmcrit),
                                         (names[1],lgy_at_mcrit),
                                         (names[2],indx_lo),
                                         (names[3],indx_hi),
                                         (names[4],tau_dep)])
            list_param_ms = [lgmcrit,lgy_at_mcrit,indx_lo,indx_hi,tau_dep]
            
        if igrp == 2:
            
            lg_qt =  numpyro.sample(names[0],dist.Normal(val0[0], scale))
            qlglgdt = numpyro.sample(names[1],dist.Normal(val0[1], scale))
            lg_drop = numpyro.sample(names[2],dist.Normal(val0[2], scale))
            lg_rejuv = numpyro.sample(names[3],dist.Normal(val0[3], scale))
            
            dict_param_q = OrderedDict([(names[0],lg_qt),
                                         (names[1], qlglgdt),
                                         (names[2],lg_drop ),
                                         (names[3],lg_rejuv)])
            list_param_q = [lg_qt, qlglgdt,lg_drop,lg_rejuv]
            
        if igrp == 3:
            Av = numpyro.sample(names[0],dist.Uniform(0.,2.))
            uv_bump_ampl = val0[1]
            plaw_slope = val0[2]
            
            dict_param_dust = OrderedDict([(names[0],Av)])
            list_param_dust = [Av]
            
            
    params = dict_param_mah
    params.update(dict_param_ms)
    params.update(dict_param_q)
    params.update(dict_param_dust)
    
    # compute SFR
    tarr = np.linspace(0.1, today_gyr, 100)
    sfh_gal = sfh_singlegal(
    tarr, list_param_mah, list_param_ms, list_param_q)
    
    # metallicity
    gal_lgmet = -2.0 # log10(Z)
    gal_lgmet_scatter = 0.2 # lognormal scatter in the metallicity distribution function

    # need age of universe when the light was emitted 
    t_obs = age_at_z(z_obs, *DEFAULT_COSMOLOGY) # age of the universe in Gyr at z_obs
    t_obs = t_obs[0] # age_at_z function returns an array, but SED functions accept a float for this argument
    
    # clear sfh in future
    sfh_gal = jnp.where(tarr<t_obs, sfh_gal, 0)
    
    # compute SED
    gal_t_table = tarr
    gal_sfr_table = sfh_gal
    sed_info = calc_rest_sed_sfh_table_lognormal_mdf(
    gal_t_table, gal_sfr_table, gal_lgmet, gal_lgmet_scatter,
    ssp_data.ssp_lgmet, ssp_data.ssp_lg_age_gyr, ssp_data.ssp_flux, t_obs)
    
    # compute dust attenuation
    wave_spec_micron = ssp_data.ssp_wave/10_000
    k = sbl18_k_lambda(wave_spec_micron,uv_bump_ampl,plaw_slope)
    dsps_flux_ratio = _frac_transmission_from_k_lambda(k,Av)
    
    sed_attenuated = dsps_flux_ratio * sed_info.rest_sed
    
    # interpolate measured
    Fobs = jnp.interp(wls, ssp_data.ssp_wave, sed_attenuated)
    
    with numpyro.plate("obs", wls.shape[0]):  # les observables sont indépendantes
        numpyro.sample('F', dist.Normal(Fobs, sigma), obs=Fobs)


In [ ]:
numpyro.render_model(galaxymodel, model_args=(jnp.array([0.]),jnp.array([1.])),render_distributions=True)

# Comvert the observed flux in a flux in rest-frame

In [ ]:
X = the_wl/(1+zobs)
Y = the_fnu*(1+zobs)
sigma_obs = 1e-6

In [ ]:
from numpyro.handlers import seed, trace, condition
# So, let's generate the data at the fiducial parameters
fiducial_model = condition(galaxymodel, params_true)
# Run the conditionned model (if not conditionned the priors are executed)
trace_data = trace(seed(fiducial_model, jax.random.PRNGKey(42))).get_trace(X, sigma=sigma_obs)

In [ ]:
#trace_data

In [ ]:
FMes_bis = trace_data['F']["value"]

In [ ]:
FMes_bis

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(X,FMes_bis,'r-',label="model (DSPS) + Dust attenuation")
ax.plot(X,Y,'b',label="data (Fors2)")
ax.legend()
ax.set_yscale('log')
ax.grid()
ax.set_title("Comparison of Fors2 Spectrum and DSPS Spectrum")

# MCMC

In [ ]:
rng_key = jax.random.PRNGKey(42)
rng_key, rng_key0, rng_key1, rng_key2 = jax.random.split(rng_key, 4)

In [ ]:
# Run NUTS.
kernel = NUTS(galaxymodel, dense_mass=True, target_accept_prob=0.9,
              init_strategy=numpyro.infer.init_to_median())
num_samples = 5_000
n_chains = 4
mcmc = MCMC(kernel, num_warmup=1_000, num_samples=num_samples,  
            num_chains=n_chains,
            chain_method='vectorized',
            progress_bar=False)
mcmc.run(rng_key, wls=X, Fobs=Y,sigma=sigma_obs,extra_fields=('potential_energy',))
mcmc.print_summary()
samples_nuts = mcmc.get_samples()

In [ ]:
az.ess(samples_nuts, relative=True)  # efficacité relative
